In [5]:
import pickle
import numpy as np

import sys
sys.path.append("../code")
import model



with open("../datamodelruns/alphaR=0.011999999999999999.pickle", "rb") as f:
    m=pickle.load(f)
    
    
print(m.chopped_data()[:,9])

[650000.         650000.         650000.         ... 703051.88720261
 703051.88720261 703051.88720261]
